# Comparing regression models

Instructions

1. Define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.

2. Use the function to check LinearRegressor,Lasso,Ridge and KNeighborsRegressor.

3. Use feature selection techniques to select subset of features to train the model with(if necessary).

4. Check and discuss the results.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
pd.options.display.max_rows = 50
## Install xlrd package to load Excel files
# conda install openpyxl
## conda install xlrd

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
## to make it possible to display multiple output inside one cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings("ignore")

In [3]:
Data_Analyzing_df= pd.read_csv("customer_case_study/Data/Data_Marketing_Customer_Analysis_Round2.csv") ## Your Code here

Data_Analyzing_df

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,1/19/11,Unemployed,F,...,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A
10906,10906,KX53892,Oregon,5259.444853,No,Basic,College,1/6/11,Employed,F,...,0.0,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A
10907,10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2/6/11,Employed,F,...,0.0,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,NaN
10908,10908,WA60547,California,11971.977650,No,Premium,College,2/13/11,Employed,F,...,4.0,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A


In [4]:
Data_Analyzing_df.drop(columns = ['Unnamed: 0'], inplace = True) #deleted the Unnamed:0 column

In [5]:
mean_customer = np.mean(Data_Analyzing_df['Customer Lifetime Value']) # Replacing null values with means of the column (for numerical columns)
Data_Analyzing_df['Customer Lifetime Value'] = Data_Analyzing_df['Customer Lifetime Value'].fillna(mean_customer)


In [6]:
mean_income = np.mean(Data_Analyzing_df['Income'])
Data_Analyzing_df['Income'] = Data_Analyzing_df['Income'].fillna(mean_income)

In [7]:
mean_complaints = np.mean(Data_Analyzing_df['Number of Open Complaints'])
Data_Analyzing_df['Number of Open Complaints'] = Data_Analyzing_df['Number of Open Complaints'].fillna(mean_complaints)

In [8]:
mean_claim = np.mean(Data_Analyzing_df['Total Claim Amount'])
Data_Analyzing_df['Total Claim Amount'] = Data_Analyzing_df['Total Claim Amount'].fillna(mean_claim)

In [9]:
Data_Analyzing_df_numerics_only = Data_Analyzing_df.select_dtypes(include=np.number)#numerical columns
Data_Analyzing_df_numerics_only

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount
0,4809.216960,48029,61,7.0,52,0.000000,9,292.800000
1,2228.525238,0,64,3.0,26,0.000000,1,744.924331
2,14947.917300,22139,100,34.0,31,0.000000,2,480.000000
3,22332.439460,49078,97,10.0,3,0.000000,2,484.013411
4,9025.067525,23675,117,NaN,31,0.384256,7,707.925645
...,...,...,...,...,...,...,...,...
10905,15563.369440,0,253,NaN,40,0.384256,7,1214.400000
10906,5259.444853,61146,65,7.0,68,0.000000,6,273.018929
10907,23893.304100,39837,201,11.0,63,0.000000,2,381.306996
10908,11971.977650,64195,158,0.0,27,4.000000,6,618.288849


## Defining X, y

In [10]:
X = Data_Analyzing_df_numerics_only.drop(columns=["Total Claim Amount"], axis = 1)
y = np.log(Data_Analyzing_df_numerics_only["Total Claim Amount"])

In [11]:
X

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies
0,4809.216960,48029,61,7.0,52,0.000000,9
1,2228.525238,0,64,3.0,26,0.000000,1
2,14947.917300,22139,100,34.0,31,0.000000,2
3,22332.439460,49078,97,10.0,3,0.000000,2
4,9025.067525,23675,117,NaN,31,0.384256,7
...,...,...,...,...,...,...,...
10905,15563.369440,0,253,NaN,40,0.384256,7
10906,5259.444853,61146,65,7.0,68,0.000000,6
10907,23893.304100,39837,201,11.0,63,0.000000,2
10908,11971.977650,64195,158,0.0,27,4.000000,6


In [12]:
y

0        5.679490
1        6.613283
2        6.173786
3        6.182113
4        6.562339
           ...   
10905    7.102005
10906    5.609541
10907    5.943605
10908    6.426956
10909    6.929242
Name: Total Claim Amount, Length: 10910, dtype: float64

## Data splitting

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test  = pd.DataFrame(X_test, columns=X.columns)

In [14]:
X_train.describe()

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies
count,8728.000000,8728.000000,8728.000000,8201.000000,8728.000000,8728.000000,8728.000000
mean,8025.739678,37593.503093,93.291247,15.105231,48.059808,0.380213,2.970784
std,6973.335781,30343.602668,34.710942,10.043650,27.969144,0.873851,2.387027
min,1898.007675,0.000000,61.000000,0.000000,0.000000,0.000000,1.000000
25%,4016.439689,0.000000,68.000000,6.000000,24.000000,0.000000,1.000000
50%,5764.823237,33889.500000,83.000000,14.000000,48.000000,0.000000,2.000000
75%,8956.200142,62198.750000,109.000000,23.000000,71.000000,0.384256,4.000000
max,83325.381190,99981.000000,298.000000,35.000000,99.000000,5.000000,9.000000


## Variance threshold method

### Univariate method

In [29]:
from sklearn.feature_selection import VarianceThreshold # It only works with numerical features


X_train = X_train.select_dtypes(include=np.number)
X_test  = X_test.select_dtypes(include=np.number)

#display(X_train)
print("Initial number of numerical columns: ",X_train.shape)
print()


selector = VarianceThreshold(threshold=100) # Default threshold value is 0
# Features with a training-set variance lower than this threshold will be removed.
selector.fit(X_train)

kept_features_indexes = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(X_train.iloc[:,kept_features_indexes].columns)

X_train = selector.transform(X_train)
X_test  = selector.transform(X_test)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)

print("Final number of numerical columns: ",X_train.shape)
print()
X_train

AttributeError: 'numpy.ndarray' object has no attribute 'select_dtypes'

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE  ## recursive feature elemination technique

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train = X_train.select_dtypes(include=np.number)
X_test  = X_test.select_dtypes(include=np.number)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test  = pd.DataFrame(X_test, columns=X.columns)
#X_train.isna().sum()
nulls = pd.DataFrame(X_train.isna().sum()).reset_index()
#nulls.head()
nulls.columns = ['Column','nas']
#nulls.head()
#nulls[nulls['nas'] > 0].head()
cols_to_drop = nulls[nulls['nas'] > 0]['Column'] # Too drastic, but made on pourpose for quick filtering (don't do this in production!!)

X_train.drop(columns=cols_to_drop, axis=1, inplace = True)
X_test.drop(columns=cols_to_drop, axis=1, inplace = True)


#display(X_train)

lm = LinearRegression()

selector = RFE(lm, n_features_to_select= 8, step = 1, verbose = 1) # Step is how many features to add or drop everytime
selector.fit(X_train, y_train)

kept_features = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(X_train.iloc[:,kept_features].columns)

X_train = selector.transform(X_train)
X_test  = selector.transform(X_test)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)

print("Final selected features: ")
display(X_train)

RFE(estimator=LinearRegression(), n_features_to_select=8, verbose=1)

Final selected features: 


,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Policy Inception,Number of Open Complaints,Number of Policies
0,4665.129599,0.0,62.0,62.0,0.0,3.0
1,10288.924950,96337.0,127.0,12.0,0.0,3.0
2,4873.436612,18866.0,126.0,62.0,0.0,1.0
3,6944.739992,0.0,68.0,31.0,0.0,2.0
4,2472.469209,63860.0,62.0,81.0,0.0,1.0
...,...,...,...,...,...,...
8723,3810.238281,0.0,108.0,57.0,0.0,1.0
8724,3815.851163,38651.0,98.0,83.0,0.0,1.0
8725,7850.590399,0.0,69.0,78.0,0.0,2.0
8726,4974.235309,0.0,70.0,74.0,0.0,3.0


# OLS

In [16]:
model=LinearRegression()
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

LinearRegression()

LinearRegression: Train -> 0.25712429341147114, Test -> 0.2381775747067033


## lasso model could drop features and make it a feature selection technique

In [17]:
from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression
model=Lasso(alpha=0.05)

model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")


Lasso(alpha=0.05)

Lasso: Train -> 0.25707960233776217, Test -> 0.23847408040569185


## Ridge

In [18]:
model=Ridge(alpha=0.1)
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")


Ridge(alpha=0.1)

Ridge: Train -> 0.25712429341146326, Test -> 0.23817757872720613


## ElasticNet

In [19]:
model=ElasticNet(alpha=0.1)
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")


ElasticNet(alpha=0.1)

ElasticNet: Train -> 0.2570795386446255, Test -> 0.23847370074120866


## Embedded Methods

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

X_train = X_train.select_dtypes(include=np.number)
X_test  = X_test.select_dtypes(include=np.number)

In [27]:
import numpy as np
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train=imp_mean.fit_transform(X_train)

In [28]:
X_train

array([[2.90393976e+03, 6.77630000e+04, 7.30000000e+01, ...,
        6.90000000e+01, 1.00000000e+00, 1.00000000e+00],
       [7.31701425e+03, 2.40830000e+04, 6.30000000e+01, ...,
        2.00000000e+00, 1.00000000e+00, 2.00000000e+00],
       [5.56888925e+03, 5.32950000e+04, 7.00000000e+01, ...,
        9.70000000e+01, 0.00000000e+00, 8.00000000e+00],
       ...,
       [7.85059040e+03, 0.00000000e+00, 6.90000000e+01, ...,
        7.80000000e+01, 0.00000000e+00, 2.00000000e+00],
       [4.97423531e+03, 0.00000000e+00, 7.00000000e+01, ...,
        7.40000000e+01, 0.00000000e+00, 3.00000000e+00],
       [3.80552095e+04, 0.00000000e+00, 1.15000000e+02, ...,
        7.80000000e+01, 0.00000000e+00, 2.00000000e+00]])